# Candy Distributor Shipping Analysis

This notebook loads the candy distributor dataset and performs exploratory data analysis (EDA) on shipping and supply-chain performance.


In [1]:
# Install any required packages (uncomment if running in a fresh environment)
# %pip install pandas numpy matplotlib seaborn

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

sns.set(style="whitegrid")


## Load candy distributor dataset

Update `data_path` below to point to your actual dataset file (for example: `data/candy_distributor.csv`).


In [ ]:
# TODO: set this to the correct relative or absolute path of your candy distributor dataset

data_path = "dataset/Candy_Sales.csv"  

if not os.path.exists(data_path):
    raise FileNotFoundError(
        f"Dataset not found at '{data_path}'. Please update 'data_path' to the correct location."
    )

# Read the dataset
df = pd.read_csv(data_path)

# Quick peek at the data
print("Shape:", df.shape)
df.head()


FileNotFoundError: Dataset not found at 'candy_distributor.csv'. Please update 'data_path' to the correct location.

## Initial data inspection

Run the following cells after successfully loading the dataset to understand its structure and basic statistics.


In [ ]:
# Column info
print("Columns:\n", df.columns.tolist())
print("\nData types:")
print(df.dtypes)

# Missing values summary
print("\nMissing values per column:")
print(df.isna().sum())


In [ ]:
# Basic descriptive statistics for numeric columns
df.describe().T
